In [100]:
import os
import json
import pandas as pd
import traceback


In [101]:
from langchain_google_genai import GoogleGenerativeAI

In [102]:
from dotenv import load_dotenv
load_dotenv()


True

In [103]:
import os 
key=os.getenv("GOOGLE_API_KEY")


In [104]:
llm= GoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0.2,
    max_output_tokens=1024,
    top_p=0.8,
    top_k=40,
    api_key=key
)

In [105]:
llm

GoogleGenerativeAI(model='gemini-1.5-flash', google_api_key=SecretStr('**********'), temperature=0.2, top_p=0.8, top_k=40, max_output_tokens=1024, client=ChatGoogleGenerativeAI(model='models/gemini-1.5-flash', google_api_key=SecretStr('**********'), temperature=0.2, top_p=0.8, top_k=40, max_output_tokens=1024, client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x0000016EF269AF60>, default_metadata=(), model_kwargs={}))

In [106]:
! pip install langchain 

In [141]:
response_json={
    "1":{
        "mcq":"multiple choice question",
        "options":{
            "a":"choose here",
            "b":"option b", 
            "c":"option c",
            "d":"option d"
        },
        "correct": "correct answer",
        
    },
    "2":{
        "mcq":" multiple choice question",
        "options":{
            "a":"option a",
            "b":"option b", 
            "c":"option c",
            "d":"option d"
        },
        "correct": "correct answer",
        
    },
    "3":{
        "mcq":" multiple choice question",
        "options":{
            "a":"option a",
            "b":"option b", 
            "c":"option c",
            "d":"option d"
        },
        "correct": "correct answer",
        
    }
}

In [127]:
template="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [142]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
prompt = PromptTemplate(
    input_variables=["text","number","subject","tone","response_json"],
    template=template
)

In [143]:
quiz_chain = LLMChain(
    llm=llm,
    prompt=prompt,
    output_key="quiz",
    verbose=True
)

In [144]:
template2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [145]:
quiz_eval_prompt = PromptTemplate(
    input_variables=["quiz","subject"],
    template=template2
)

In [146]:
chain2=LLMChain(
    llm=llm,
    prompt=quiz_eval_prompt,
    output_key="quiz_eval", 
    verbose=True
)

In [147]:
fchain= SequentialChain(
    chains=[quiz_chain,chain2],
    input_variables=["text","number","subject","tone","response_json"],
    output_variables=["quiz","quiz_eval"],
    verbose=True
)

In [148]:
file_path = r"D:\mcqgen\data.txt"

In [149]:
with open(file_path, "r") as file:
    text = file.read()

In [154]:
# ...existing code...
number =4
subject = "machine learning"
tone = "simple"
response = fchain({
    "text": text,
    "number": number,
    "subject": subject,
    "tone": tone,
    "response_json": json.dumps(response_json)
})
# ...existing code...



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.[8][9] The synonym self-teaching computers was also used in this time period.[10][11]

Although the earliest machine learning model was introduced in the 1950s when Arthur Samuel invented a program that calculated the winning chance in checkers for each side, the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[12] In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells.[13] Hebb's model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons used b

In [156]:
quiz=response.get("quiz")
quiz_eval=response.get("quiz_eval") 

In [157]:
print(response.get("quiz"))

```json
{
  "1": {
    "mcq": "Who coined the term 'machine learning'?",
    "options": {
      "a": "Donald Hebb",
      "b": "Arthur Samuel",
      "c": "Tom M. Mitchell",
      "d": "Alan Turing"
    },
    "correct": "b"
  },
  "2": {
    "mcq": "Which of the following is NOT a primary objective of modern-day machine learning, as described in the text?",
    "options": {
      "a": "Classifying data based on developed models",
      "b": "Making predictions for future outcomes based on models",
      "c": "Developing entirely new programming languages for AI",
      "d": "Both a and b"
    },
    "correct": "c"
  },
  "3": {
    "mcq": "What early device, developed by Raytheon, used reinforcement learning to analyze signals?",
    "options": {
      "a": "A perceptron",
      "b": "Cybertron",
      "c": "An expert system",
      "d": "A generalized linear model"
    },
    "correct": "b"
  },
  "4": {
    "mcq": "According to the text, what significant shift occurred in machine le